# PA 8: Association Rule Mining

For this assignment, we will implement the apriori algorithm for association rule mining to both discover association rules in a dataset and compute their "interestingness" statistics.

## Step 1 - Apriori Algorithm

The first step is to implement the Apriori algorithm for use on further datasets. To do so, we will first borrow several dataset creation and cleaning functions from previous assignments, listed below.

In [1]:
def read_data(filename):
    f = open(filename, 'r')
    text = f.read()
    f.close()
    return text

def create_dataset(data):
    data_r = data.splitlines()
    dataset = []
    for line in data_r:
        instance = line.split(',')
        dataset.append(instance)
    return dataset

We will then need several helper functions in order to follow the apriori algorithm. Their descriptions are as follows:

* `compute_unique_values()`
    * **Params**:
        * `table` - The table to compute unique values for
    * **Returns**:
        * A list of all unique elements in the dataset, sorted
        
* `compute_subsets()`
    * **Params**:
        * `itemset` - The L_k-1 itemsets to compute L_k subsets for
    * **Returns**:
        * A list of all itemsets such that, for two elements of L_k-1 called A and B, the itemset is A + B[ -1 ] if all but the last element of A and B are identical
        
* `get_permutations()`
    * **Params**:
        * `itemset` - The set of elements to compute k-1 permutations for
    * **Returns**:
        * A list of all permutations of the data with length k-1
        
* `prepend_headers()`
    * **Params**:
        * `itemset` - A table of data to prepend headers to
        * `headers` - The list of headers for each column to prepend to elements of `itemset`
    * **Returns**:
        * The data table where each element is prepended with the header for that column
        
* `prune_absent_subset()`
    * **Params**:
        * `Lk` - The list L_k-1 in the apriori algorithm
        * `Ck` - The candidate set for L_k to be pruned
    * **Returns**:
        * A list of all elements of `Ck` for which all k-1 subsets exist within `Lk`
        * In other words, `Ck` pruned so that all elements for which a subset of the element does not exist in L_k-1 are removed
        
* `calculate_support()`
    * **Params**:
        * `transactions` - The list of all transactions in the data
        * `element` - The element to calculate support for
    * **Returns**:
        * The support for the given element in the data, described by the function $\frac{count(S)}{N}$
        
* `prune_support()`
    * **Params**:
        * `transactions` - The list of all transactions in the data
        * `Ck` - The candidate set to prune elements from
        * `minsupport` - The minimum support value that an element can have before it is pruned
    * **Returns**:
        * `Ck` pruned such that all elements with a support of less than `minsupport` are removed
        
* `generate_rhs()`
    * **Params**:
        * `S` - The itemset to generate all possible RHS subsets for
        * `k` - the length of generated RHS subsets; used for recursion
    * **Returns**:
        * A list of all possible RHS subsets for the given itemset
        
* `generate_rules()`
    * **Params**:
        * `L` - The superset of all possible itemsets generated from the apriori algorithm
    * **Returns**:
        * A list of rules in the form { "lhs" : [...], "rhs" : [...] }
        
* `prune_confidence()`
    * **Params**:
        * `transactions` - The data table of all transactions
        * `rules` - A list of rules in the above format to be pruned
        * `minconf` - The minimum confidence value a rule can have before it is pruned, calculated using the formula $\frac{count(S)}{count(LHS)}$
    * **Returns**:
        * A list of all rules, pruned such that all rules with a confidence below `minconf` are pruned

In [2]:
def compute_unique_values(table):
    I = set()
    for row in table:
        for elem in row:
            I.add(elem)
    return list(I)


def compute_subsets(itemset):
    L = []
    for a in range(len(itemset)):
        for b in range(a+1, len(itemset)):
            A = itemset[a]
            B = itemset[b]
            if A != B and A[:-1] == B[:-1]:
                AB = A + [B[-1]]
                L.append(AB)
    return L


def get_permutations(itemset):
    permutations = []
    for i in range(len(itemset)):
        permutation = itemset[:]
        del permutation[i]
        permutations.append(permutation)
    return permutations


def prepend_headers(itemset, headers):
    prepended = []
    for row in itemset:
        prepended.append([headers[i] + "=" + row[i] for i in range(len(headers))])
    return prepended


def prune_absent_subset(Lk, Ck):
    Ck_pruned = []
    for elem in Ck:
        prune = False
        for permutation in get_permutations(elem):
            if permutation not in Lk:
                prune = True
                break
        if not prune:
            Ck_pruned.append(elem)
    return Ck_pruned


def calculate_support(transactions, element):
    count = 0
    for item in transactions:
        supported = True
        for e in element:
            if e not in item:
                supported = False
        if supported:
            count += 1
    return count / len(transactions)


def prune_support(transactions, Ck, minsupport):
    Ck_pruned = []
    for element in Ck:
        supp = calculate_support(transactions, element)
        if supp >= minsupport:
            Ck_pruned.append(element)
    return Ck_pruned


def generate_rhs(S, k):
    if k == 1:
        return [[x] for x in S]
    else:
        rhs = []
        for i in range(len(S) + 1 - k):
            rhs += [[S[i]] + x for x in generate_rhs(S[i+1:], k-1)]
        return rhs


def generate_rules(L):
    rules = []
    for subset in L:
        for S in subset:
            for i in range(1, len(S)):
                rhs_i = generate_rhs(S, i)
                for rhs in rhs_i:
                    lhs = []
                    for elem in S:
                        if elem not in rhs:
                            lhs.append(elem)
                    
                    rule = {"lhs":lhs, "rhs":rhs}
                    rules.append(rule)
    return rules


def prune_confidence(transactions, rules, minconf):
    pruned = []
    for rule in rules:
        count_s = 0
        count_l = 0
        lhs = rule["lhs"]
        rhs = rule["rhs"]
        for row in transactions:
            both = True
            left = True
            for elem in lhs:
                if elem not in row:
                    left = False
                    both = False
            if left == True:
                count_l += 1
                for elem in rhs:
                    if elem not in row:
                        both = False
                if both == True:
                    count_s += 1
        conf = count_s / count_l
        if conf >= minconf:
            pruned.append(rule)
    return pruned

After computing the rules using apriori, we will also need to compute the statistics Support, Confidence, and Lift for each value. Here we will define a function to calculate these values:

* `calculate_stats()`
    * **Params**:
        * `transactions` - The list of all transactions in the dataset
        * `rules` - The list of all rules to calculate statistics for
    * **Returns**:
        * A 2D array of all of the rules and their corresponding statistics, such that each row takes the form `\[rule, support, confidence, lift\]
        * Lift is calculated using the function $\frac{support(S)}{support(LHS)\times support(RHS)}$

In [3]:
def calculate_stats(transactions, rules):
    stats = []
    for rule in rules:
        count_l = 0
        count_r = 0
        count_s = 0
        lhs = rule["lhs"]
        rhs = rule["rhs"]
        for row in transactions:
            left = True
            right = True
            for elem in lhs:
                if elem not in row:
                    left = False
            for elem in rhs:
                if elem not in row:
                    right = False
            if left:
                count_l += 1
            if right:
                count_r += 1
            if left and right:
                count_s += 1
        support_l = count_l / len(transactions)
        support_r = count_r / len(transactions)
        support = count_s / len(transactions)
        confidence = count_s / count_l
        lift = support / (support_l * support_r)
        stats.append([rule, support, confidence, lift])
    return stats

Finally, we can define our apriori algorithm, as well as a function to pretty print the returned rules and summary statistics.

In [24]:
from tabulate import tabulate

def apriori(transactions, minsupport, minconf, headers=[]):
    if headers != []:
        transactions = prepend_headers(transactions, headers)
    supported = []
    I = sorted(compute_unique_values(transactions))
    L = [[x] for x in I]
    L = prune_support(transactions, L, minsupport)
    while L != []:
        Ck = compute_subsets(L)
        Ck = prune_absent_subset(L, Ck)
        L = prune_support(transactions, Ck, minsupport)
        if L != []:
            supported.append(L)
    rules = generate_rules(supported)
    rules = prune_confidence(transactions, rules, minconf)
    stats = calculate_stats(transactions, rules)
    return stats

def pprint(stats):
    pretty_stats = []
    for rule in stats:
        string = ""
        lhs = rule[0]["lhs"]
        rhs = rule[0]["rhs"]
        string += lhs[0]
        if len(lhs) > 1:
            for i in range(1, len(lhs)):
                string += " AND " + lhs[i]
        string += " => " + rhs[0]
        if len(rhs) > 1:
            for i in range(1, len(rhs)):
                string += " AND " + rhs[i]
        pretty_stats.append([string]+rule[1:])
    header = ["association rule", "support", "confidence", "lift"]
    print(tabulate(pretty_stats, headers=header))

In order to test this function, we will run it on two datasets used in class. The first is the interview dataset:

In [25]:
header = ["level", "lang", "tweets", "phd", "interviewed_well"]
table = [
        ["Senior", "Java", "no", "no", "False"],
        ["Senior", "Java", "no", "yes", "False"],
        ["Mid", "Python", "no", "no", "True"],
        ["Junior", "Python", "no", "no", "True"],
        ["Junior", "R", "yes", "no", "True"],
        ["Junior", "R", "yes", "yes", "False"],
        ["Mid", "R", "yes", "yes", "True"],
        ["Senior", "Python", "no", "no", "False"],
        ["Senior", "R", "yes", "no", "True"],
        ["Junior", "Python", "yes", "no", "True"],
        ["Senior", "Python", "yes", "yes", "True"],
        ["Mid", "Python", "no", "yes", "True"],
        ["Mid", "Java", "yes", "no", "True"],
        ["Junior", "Python", "no", "yes", "False"]
    ]

We will use a minsupport of 0.25 and a minconf of 0.75

In [26]:
pprint(apriori(table, 0.25, 0.75, headers=header))

association rule                                  support    confidence     lift
----------------------------------------------  ---------  ------------  -------
interviewed_well=False => tweets=no              0.285714      0.8       1.6
level=Mid => interviewed_well=True               0.285714      1         1.55556
phd=no => interviewed_well=True                  0.428571      0.75      1.16667
tweets=yes => interviewed_well=True              0.428571      0.857143  1.33333
lang=R => tweets=yes                             0.285714      1         2
phd=no AND tweets=yes => interviewed_well=True   0.285714      1         1.55556


Next, we will run our algorithm on the market basket analysis transactions:

In [27]:
transactions = [
    ["b", "c", "m"],
    ["b", "c", "e", "m", "s"],
    ["b"],
    ["c", "e", "s"],
    ["c"],
    ["b", "c", "s"],
    ["c", "e", "s"],
    ["c", "e"]
]

We will use the same values of 0.25 and 0.75 for minsupport and minconf, respectively.

In [28]:
pprint(apriori(transactions, 0.25, 0.75))

association rule      support    confidence      lift
------------------  ---------  ------------  --------
b => c                  0.375          0.75  0.857143
m => b                  0.25           1     2
e => c                  0.5            1     1.14286
m => c                  0.25           1     1.14286
s => c                  0.5            1     1.14286
s => e                  0.375          0.75  1.5
e => s                  0.375          0.75  1.5
c AND m => b            0.25           1     2
b AND m => c            0.25           1     1.14286
m => b AND c            0.25           1     2.66667
b AND s => c            0.25           1     1.14286
e AND s => c            0.375          1     1.14286
c AND s => e            0.375          0.75  1.5
c AND e => s            0.375          0.75  1.5
s => c AND e            0.375          0.75  1.5
e => c AND s            0.375          0.75  1.5


## Step 2 - Titanic Dataset

For this step, we will run our apriori algorithm over the titanic dataset. First, we will need to retrieve and clean the data for use in the algorithm.

In [29]:
titanic_data = create_dataset(read_data("titanic_data.txt"))

Next, we can simply plug our dataset, along with a list of headers, into our apriori algorithm and have it pretty print the rules and associated statistics. Since this is a much larger dataset, we will restrict it to the most interesting rules, and will therefore set our minsupport as 0.3 and our minconf as 0.9, ensuring that each rule we produce is well supported in the dataset and we are reasonably confident it is accurate.

In [30]:
titanic_headers = ["class", "age", "sex", "survived"]
pprint(apriori(titanic_data, 0.3, 0.9, headers=titanic_headers))

association rule                                            support    confidence     lift
--------------------------------------------------------  ---------  ------------  -------
class=third => age=adult                                   0.40209       1         1.0521
sex=female => age=adult                                    0.757383      0.963027  1.0132
survived=yes => age=adult                                  0.653339      0.965101  1.01539
class=third => sex=female                                  0.39164       0.974011  1.23847
survived=yes => sex=female                                 0.619718      0.915436  1.16399
class=third AND sex=female => age=adult                    0.39164       1         1.0521
age=adult AND class=third => sex=female                    0.39164       0.974011  1.23847
class=third => age=adult AND sex=female                    0.39164       0.974011  1.28602
class=third AND survived=yes => age=adult                  0.30577       1         1.0521
sex

From these rules, we can see some interesting associations. Some make sense, such as being an adult and surviving implies you are a female with very good support and confidence. Some are more odd though, such as the fact that if you are a third class female passenger, our rules imply you are an adult with a confidence of 1, meaning there were no instances of third class female children at all, which is interesting in tandem with the fact that being third class is heavily correlated with being female, meaning there were very few third class male passengers to begin with. 

## Step 3 - Mushroom Dataset

For this step, we will once again run our apriori algorithm, this time over the agaricus-lepiota dataset. This dataset contains information about different types of mushrooms and whether they are edible or posionous. The breakdown of all attributes for the dataset is as follows, copied from the PA description:

1. The class label edible=e, poisonous=p
2. cap-shape bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
3. cap-surface fibrous=f, grooves=g, scaly=y, smooth=s
4. cap-color brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
5. bruises? bruises=t, no=f
6. odor almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
7. gill-attachment attached=a, descending=d, free=f, notched=n
8. gill-spacing close=c, crowded=w, distant=d
9. gill-size broad=b, narrow=n
10. gill-color black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y
11. stalk-shape enlarging=e, tapering=t
12. stalk-root bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r
13. stalk-surface-above-ring fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-surface-below-ring fibrous=f,scaly=y,silky=k,smooth=s
15. stalk-color-above-ring brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
16. stalk-color-below-ring brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
17. veil-type partial=p, universal=u
18. veil-color brown=n, orange=o, white=w, yellow=y
19. ring-number none=n, one=o, two=t
20. ring-type cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
21. spore-print-color black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
22. population abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
23. habitat grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

As before, our first step is to grab the dataset.

In [31]:
mushroom_data = create_dataset(read_data("agaricus-lepiota.txt"))

mushroom_headers = ["class", "cap-shape", "cap-surface", "cap-color", "bruises?", "odor", "gill-attachment", \
          "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring", \
          "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", \
          "veil-color", "ring-number", "ring-type", "spore-print-color", "population", "habitat"]

Because of the size of this data, we will need to use feature selection to prune the data to only a subset of the given attributes. For this purpose, we will write the helper function `feature_selection()`

In [32]:
def feature_selection(dataset, features, headers):
    pruned = []
    pruned_headers = []
    for row in dataset:
        pruned_row = []
        for i in features:
            pruned_row.append(row[i])
        pruned.append(pruned_row)
    for i in features:
        pruned_headers.append(headers[i])
    return pruned, pruned_headers

For the purpose of this step, we will choose the following attributes to keep:

* edible
* cap-shape
* cap-surface
* cap-color
* odor
* gill-attachment
* gill-size
* gill-color
* ring-number
* ring-type
* spore-print
* population
* habitat

In [33]:
mushroom_data_pruned, mushroom_headers_pruned = feature_selection(mushroom_data, [0, 1, 2, 3, 5, 6, 8, 9, 18, 19, 20, 21, 22], mushroom_headers)

We will now run the apriori algorithm over the pruned dataset. Since this is a huge dataset with many attributes, we will want to restrict our rule generation significantly. Therefore, we will only accept rules with a support of at least 0.6, and a confidence of 0.95, to avoid generating a huge number of rules.

In [38]:
pprint(apriori(mushroom_data_pruned, 0.6, 0.95, headers=mushroom_headers_pruned))

association rule                                      support    confidence      lift
--------------------------------------------------  ---------  ------------  --------
class=e => gill-attachment=f                         0.618001      1         1.0032
class=e => ring-number=o                             0.609497      0.986239  1.01427
gill-size=b => gill-attachment=f                     0.872077      0.996356  0.999544
ring-number=o => gill-attachment=f                   0.97236       1         1.0032
gill-attachment=f => ring-number=o                   0.97236       0.975471  1.0032
ring-type=p => gill-attachment=f                     0.618001      1         1.0032
gill-size=b => ring-number=o                         0.847626      0.968421  0.995949
class=e AND ring-number=o => gill-attachment=f       0.609497      1         1.0032
class=e AND gill-attachment=f => ring-number=o       0.609497      0.986239  1.01427
class=e => gill-attachment=f AND ring-number=o       0.609497     